## Read in text file

In [ ]:
with open("litigation.txt", "r") as f:
    raw_text_lines = f.readlines()

## Parse out separate cases

In [ ]:
import re
date_re = re.compile(r"(January|February|March|April|May|June|July|August|September|October|November|December)\s[1-3]?[0-9]+,\s[0-9]+")

In [ ]:
all_heads = []
body_indices = []
header = ""
binary = 0
start = 0
for i, l in enumerate(raw_text_lines):
    if l.startswith("        "):
        if binary == 0:
            if l.strip()[:2].isupper() and l.strip()[:2].isalpha():
                binary = 1
                body_indices.append((start, i))

        if i < len(raw_text_lines):
            if re.search(date_re, l) and re.search(date_re, raw_text_lines[i+1]) == None:
                header += l
                all_heads.append(header)
                header = ""
                binary = 0
                start = i + 1
            
        if binary == 1:
            header += l

In [ ]:
head_body = []

for i, pair in enumerate(body_indices[1:]):
    head_body.append((all_heads[i], ''.join(raw_text_lines[pair[0]:pair[1]])))

In [ ]:
print(len(head_body))

In [ ]:
print(head_body[0][0])

In [ ]:
print(head_body[0][1])

## Extract information

In [ ]:
from nltk.tag import pos_tag
from nltk import word_tokenize
current = head_body[100][1]
outcome = current[current.find("OUTCOME:"):].split("\n")[0]
pos_outcome = pos_tag(word_tokenize(outcome)[2:])
print(pos_outcome)

## Patent co-referencing

In [ ]:
pairs = []
pattern = re.compile(r"Patent No\. (?P<number>[0-9\,]+)\s\((?:the\s)?'(?P<alt>[0-9]+) patent\)")
for x in head_body:
    pairs.extend(re.findall(pattern, x[1]))

In [ ]:
print(len(pairs))
print(pairs[0])

In [ ]:
id_dict = {int(k.replace(",", "")): int(v.replace(",", "")) for (k, v) in pairs}
print(id_dict[5337753])